In [ ]:
import string
import pandas as pd
import nltk
#import spacy
from nltk.corpus import stopwords
stop = stopwords.words('english')

import tensorflow as tf

import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import re
import numpy as np
import tqdm

from transformers.models.bert.modeling_bert import BertModel,BertForMaskedLM
from simpletransformers.language_representation import RepresentationModel

In [ ]:
#python -m nltk.downloader stopwords
!pip install tqdm
#!pip install bert-for-tf2
#!pip install transformers
#!pip install torch
#!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!cd nltk_data/corpora/
#!unzip stopwords.zip
!ls

nltk_data


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
#preprocessing the data 
TAG_RE = re.compile(r'<[^>]+>')

def preprocessing(df):
   
    #remove stop words
    df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


                                                            
    #remove puncaution marks 
    df["review"] = df["review"].str.replace('[{}]'.format(string.punctuation), '')
                                                            
                                                            
    #converting to lower case
    df["review"] = df["review"].str.lower()
                                              
    def remove_tags(text):
         
      return TAG_RE.sub('', text)

    def remove_special_characters(text, remove_digits=True):
        pattern=r'[^a-zA-z0-9\s]'
        text=re.sub(pattern,'',text)
        return text
        #Apply function on review column
    df['review']=df['review'].apply(remove_special_characters)
    df['review'] = df['review'].apply(remove_tags)

    
preprocessing(df)
                                                            

<ipython-input-39-6cd3bd798a06>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df["review"] = df["review"].str.replace('[{}]'.format(string.punctuation), '')


In [ ]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,a wonderful little production br br the filmin...,positive
2,i thought wonderful way spend time hot summer ...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive


In [ ]:
df['review'].iloc[2]

'thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy plot simplistic dialogue witty characters likable even well bread suspected serial killer may disappointed realize match point 2 risk addiction thought proof woody allen still fully control style many us grown lovebr br id laughed one woodys comedies years dare say decade ive never impressed scarlet johanson managed tone sexy image jumped right average spirited young womanbr br may crown jewel career wittier devil wears prada interesting superman great comedy go see friends'

In [ ]:
# Load Tokenizer and Model
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

#loading bert tokenizer
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
tokenized_reviews[1:10]

[['wonderful',
  'little',
  'production',
  'br',
  'br',
  'filming',
  'technique',
  'una',
  '##ss',
  '##uming',
  'old',
  '##time',
  '##bb',
  '##c',
  'fashion',
  'gives',
  'comforting',
  'sometimes',
  'discomfort',
  '##ing',
  'sense',
  'realism',
  'entire',
  'piece',
  'br',
  'br',
  'actors',
  'extremely',
  'well',
  'chosen',
  'michael',
  'sheen',
  'got',
  'polar',
  '##i',
  'voices',
  'pat',
  'truly',
  'see',
  'seam',
  '##less',
  'editing',
  'guided',
  'references',
  'williams',
  'diary',
  'entries',
  'well',
  'worth',
  'watching',
  'terrific',
  '##ly',
  'written',
  'performed',
  'piece',
  'master',
  '##ful',
  'production',
  'one',
  'great',
  'masters',
  'comedy',
  'life',
  'br',
  'br',
  'realism',
  'really',
  'comes',
  'home',
  'little',
  'things',
  'fantasy',
  'guard',
  'rather',
  'use',
  'traditional',
  'dream',
  'techniques',
  'remains',
  'solid',
  'disappears',
  'plays',
  'knowledge',
  'senses',
  'part

In [ ]:
model=RepresentationModel(model_type="bert",model_name="bert-base-uncased", use_cuda=True)
vectorized_senteces=model.encode_sentences(list(df["review"]),combine_strategy=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
vectorized_senteces

array([[[ 5.23883663e-02, -5.66922128e-01, -1.17071472e-01, ...,
         -3.09837818e-01,  7.38367617e-01,  1.04891032e-01],
        [-7.71952420e-02,  2.20062733e-01, -1.07361446e-03, ...,
          1.14167430e-01,  7.26827919e-01, -2.76951551e-01],
        [-6.64291382e-01, -1.58171449e-02,  4.25968617e-01, ...,
          1.41838685e-01,  9.22016442e-01, -1.95172019e-02],
        ...,
        [-2.57577121e-01,  1.20168567e+00, -2.40622344e-03, ...,
          3.40075433e-01,  4.40398097e-01,  1.28956899e-01],
        [-5.91713667e-01,  6.74769521e-01, -6.13256335e-01, ...,
          2.18867827e-02,  5.40603220e-01,  6.85783446e-01],
        [ 1.21943161e-01, -2.53836513e-01,  5.94137609e-02, ...,
          4.70073670e-01,  4.06410486e-01, -1.84780821e-01]],

       [[-1.21517144e-01, -1.24739921e-02,  4.52362038e-02, ...,
         -3.49425226e-01,  3.92431438e-01,  1.08350873e-01],
        [ 1.92627311e-01,  6.26040339e-01,  6.06513679e-01, ...,
         -3.19178067e-02,  3.46468151e

In [ ]:
# Save sentiments as dependent variable y
y = df['sentiment']

# Set 1 for positive reviews and 0 for negative reviews
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [ ]:
# Tokenize all reviews
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))
tokenized_reviews = [tokenize_reviews(review) for review in review]

# Create list of list with review, sentiment, and length of review for each entry
#review = [reviews for reviews in list(df['review'])]
reviews_with_len = [[review, y[i], len(review)]
 
                     for i, review in tqdm.tqdm(enumerate(tokenized_reviews))]

50000it [00:00, 1140041.53it/s]


In [ ]:
import random
# Shuffle dataset
random.shuffle(reviews_with_len)

# Sort by length of review
reviews_with_len.sort(key=lambda x: x[2])

# Drop review
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in tqdm.tqdm(reviews_with_len)]

100%|██████████| 50000/50000 [00:00<00:00, 1299302.38it/s]


In [ ]:
sorted_reviews_labels

[([5896, 2466, 6752], 0),
 ([3191, 2338, 5293, 3185], 0),
 ([3185, 6659, 2204, 3896], 0),
 ([2307, 3185, 2919, 2800, 2188, 2678], 1),
 ([23873, 3993, 11259, 2172, 2172, 14888], 0),
 ([9278, 2028, 2130, 7922, 12635, 2305], 0),
 ([3246, 2177, 16587, 2196, 25372, 2015], 0),
 ([8235, 3048, 4616, 3419, 2457, 27727, 2848, 16133], 1),
 ([2204, 2143, 6057, 2664, 2143, 2204, 8471, 3152], 1),
 ([7918, 14674, 7662, 6581, 2143, 3084, 17160, 2450], 1),
 ([2113, 2066, 3185, 2092, 2196, 2131, 5458, 3666], 1),
 ([2302, 4797, 5409, 3185, 2412, 2464, 6057, 5875, 2081], 0),
 ([6283, 5409, 3185, 2412, 10597, 21985, 2393, 2919, 24669], 0),
 ([11861, 21442, 6895, 3238, 2210, 22759, 6198, 3185, 12487], 0),
 ([3078, 5436, 18098, 9581, 2854, 3257, 24667, 2099, 7613], 0),
 ([7615, 5236, 3185, 3772, 2779, 4788, 9000, 3168, 13558], 0),
 ([2412, 5506, 5353, 14711, 9120, 2143, 6581, 4569, 4756, 3371], 1),
 ([15764, 3185, 2544, 8429, 24905, 17988, 7659, 2498, 13842, 5312], 1),
 ([4602, 3185, 2412, 2517, 2412, 3773, 

**Model Building**

In [ ]:
# Create Tensorflow datasets
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

# Define batch size and cut datasets by batch size
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
#
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

**Create the Model**

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
# Hyperparameters
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 2

# Build Model
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

# Compile Model
text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)


Epoch 1/2
1407/1407 [==============================] - 446s 308ms/step - loss: 0.3116 - accuracy: 0.8628
Epoch 2/2
1407/1407 [==============================] - 18s 13ms/step - loss: 0.1183 - accuracy: 0.9572


In [ ]:
text_model.evaluate(test_data)

156/156 [==============================] - 7s 36ms/step - loss: 0.3786 - accuracy: 0.8834


[0.3786437213420868, 0.8834134340286255]

In [ ]:
# Test model on two individual reviews
test_reviews = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']
test_reviews_with_len = [[tokenize_reviews(test_reviews[0]), 
                          1, 
                          len(tokenize_reviews(test_reviews[0]))],
                         [tokenize_reviews(test_reviews[1]), 
                          0,  
                          len(tokenize_reviews(test_reviews[1]))]]
test_sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in tqdm.tqdm(test_reviews_with_len)]

predict_input = tf.data.Dataset.from_generator(lambda: test_sorted_reviews_labels, output_types=(tf.int32, tf.int32))
BATCH_SIZE = 2
test_batched_dataset = predict_input.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

# Get model prediction
text_model.predict(test_batched_dataset)

100%|██████████| 2/2 [00:00<00:00, 4662.93it/s]


1/1 [==============================] - 0s 270ms/step


array([[0.9996767 ],
       [0.00493998]], dtype=float32)